# Cleaning the Toronto postal codes database

## Load the csv file

The Wikipedia table containing the postal codes of Toronto Area has been converted to a CSV file, which can readily be loaded into a Pandas dataframe.

In [117]:
import pandas as pd
raw_data = pd.read_csv('toronto_postal_codes.csv')
raw_data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [118]:
print(raw_data.shape)

(288, 3)


## Drop the cells where Borough is not assigned

In [119]:
toronto_boroughs = raw_data.drop(raw_data.loc[raw_data['Borough']=='Not assigned'].index, inplace=False).reset_index(drop=True)
toronto_boroughs.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [120]:
print(toronto_boroughs.shape)

(211, 3)


## Assign name of Borough to unnamed neighbourhoods

Check unnamed neighbourhoods

In [121]:
toronto_boroughs.loc[toronto_boroughs['Neighbourhood'] == 'Not assigned']

,Postcode,Borough,Neighbourhood
6,M7A,Queen's Park,Not assigned


Perform substitution

In [122]:
toronto_boroughs.loc[toronto_boroughs['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = toronto_boroughs.loc[toronto_boroughs['Neighbourhood'] == 'Not assigned', 'Borough']

Check result

In [123]:
toronto_boroughs.iloc[6,:]

Postcode                  M7A
Borough          Queen's Park
Neighbourhood    Queen's Park
Name: 6, dtype: object

The substitution worked correctly.

## Group neighbourhoods by postcode

Add a comma and a space at the end of each Neighbourhood cell.

In [124]:
toronto_boroughs['Neighbourhood'] =  toronto_boroughs['Neighbourhood'].astype(str) + ', '
toronto_boroughs.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,"Parkwoods,"
1,M4A,North York,"Victoria Village,"
2,M5A,Downtown Toronto,"Harbourfront,"
3,M5A,Downtown Toronto,"Regent Park,"
4,M6A,North York,"Lawrence Heights,"


Group the neighbourhoods by their shared postcode.

In [125]:
aggregation_functions = {'Borough': 'first', 'Neighbourhood': 'sum'}
df_new = toronto_boroughs.groupby(toronto_boroughs['Postcode']).aggregate(aggregation_functions).reset_index()
df_new.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern,"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union,"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill,"
3,M1G,Scarborough,"Woburn,"
4,M1H,Scarborough,"Cedarbrae,"


Finally, delete the trailing comma and space from each Neighbourhood list.

In [126]:
df_new['Neighbourhood'] = df_new['Neighbourhood'].str.rstrip(', ')
df_new.head(11)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Save the clean database for use in the next steps of the project.

In [127]:
df_new.to_pickle('toronto_postcode_clean.pkl')

Finally, check dataset dimensions.

In [128]:
df_new.shape

(103, 3)